In [20]:
%matplotlib inline

In [77]:
from apiclient import discovery
from apiclient import errors
from httplib2 import Http
from oauth2client import file, client, tools
import base64
from bs4 import BeautifulSoup
import re
import time
import dateutil.parser as parser
from datetime import datetime
import datetime
import csv
from oauth2client.client import AccessTokenCredentials
import httplib2

In [78]:
user_id =  'me'
label_id_one = 'INBOX'
label_id_two = 'UNREAD'
credentials = AccessTokenCredentials('ya29.GlwFB8oa3Uk0qpb23Ebnv0pdcuBgvOGrif2IcJtdyUh_R6DuUhEZ3V-ps1PFbYUD3HbqO3J5qcY1VCzkY7-0I2Yp5RKwQ3nXFY5r4XDN5anzkI2GoS-S1MRL_3uQ0w','my-user-agent/1.0')
http = httplib2.Http()
http = credentials.authorize(http)
GMAIL = discovery.build('gmail', 'v1', http=http)
unread_msgs = GMAIL.users().messages().list(userId='me',labelIds=[label_id_one, label_id_two]).execute()
mssg_list = unread_msgs['messages']
print ("Total unread messages in inbox: ", str(len(mssg_list)))

Total unread messages in inbox:  13


In [79]:
final_list = [ ]


for mssg in mssg_list:
    temp_dict = { }
    m_id = mssg['id'] # get id of individual message
    message = GMAIL.users().messages().get(userId=user_id, id=m_id).execute() # fetch the message using API
    payld = message['payload'] # get payload of the message 
    headr = payld['headers'] # get header of the payload


    for one in headr: # getting the Subject
        if one['name'] == 'Subject':
            msg_subject = one['value']
            temp_dict['Subject'] = msg_subject
        else:
            pass


    for two in headr: # getting the date
        if two['name'] == 'Date':
            msg_date = two['value']
            date_parse = (parser.parse(msg_date))
            m_date = (date_parse.date())
            temp_dict['Date'] = str(m_date)
        else:
            pass

    for three in headr: # getting the Sender
        if three['name'] == 'From':
            msg_from = three['value']
            temp_dict['Sender'] = msg_from
        else:
            pass

    temp_dict['Snippet'] = message['snippet'] # fetching message snippet


    try:
        # Fetching message body
        mssg_parts = payld['parts'] # fetching the message parts
        part_one  = mssg_parts[0] # fetching first element of the part 
        part_body = part_one['body'] # fetching body of the message
        part_data = part_body['data'] # fetching data from the body
        #clean_one = part_data.replace("-","+") # decoding from Base64 to UTF-8
        #clean_one = clean_one.replace("_","/") # decoding from Base64 to UTF-8
        clean_two = base64.b64decode (bytes(part_data, 'UTF-8')) # decoding from Base64 to UTF-8
        soup = BeautifulSoup(clean_two , "lxml" )
        mssg_body = soup.body()
        # mssg_body is a readible form of message body
        # depending on the end user's requirements, it can be further cleaned 
        # using regex, beautiful soup, or any other method
        # mssg_body = mssg_body.replace("-", " ")
        temp_dict['Message_body'] = mssg_body

    except Exception as e: 
        print(e)


    print (temp_dict)
    final_list.append(temp_dict) # This will create a dictonary item in the final list
    print(type(final_list))

    # This will mark the messagea as read
    #GMAIL.users().messages().modify(userId=user_id, id=m_id,body={ 'removeLabelIds': ['UNREAD']}).execute() 




print ("Total messaged retrived: ", str(len(final_list)))


#exporting the values as .csv
with open('CSV_NAME.csv', 'w', encoding='utf-8', newline = '') as csvfile: 
    fieldnames = ['Sender','Subject','Date','Snippet','Message_body']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter = ',')
    writer.writeheader()
    for val in final_list:
       writer.writerow(val)

{'Subject': 'CMPE272 Class Lecture today at 5:00 PM', 'Date': '2019-05-07', 'Sender': 'wamique Ansari <wamique.ansari@gmail.com>', 'Snippet': 'Dear Class, Today&#39;s class lecture will be at 5:00PM. Regards TA', 'Message_body': [<p>Dear Class,

Today's  class lecture will be at 5:00PM.

Regards
TA
</p>]}
<class 'list'>
{'Subject': 'Meet SJSU alumni and get a chance for 1:1 mentor ship', 'Date': '2019-05-07', 'Sender': 'wamique Ansari <wamique.ansari@gmail.com>', 'Snippet': 'Dear Student, Alumni Connection - Meet SJSU alumni and get a chance for mentor ship on 06/20/2019 at 11:00 AM. Best Regards Wamique', 'Message_body': [<p>Dear Student,

Alumni Connection - Meet SJSU alumni and get a chance for mentor ship on
06/20/2019 at 11:00 AM.

Best Regards
Wamique
</p>]}
<class 'list'>
{'Subject': 'Tech Talk - Virtual Reality', 'Date': '2019-05-07', 'Sender': 'wamique Ansari <wamique.ansari@gmail.com>', 'Snippet': 'Dear Student, There is a Tech Talk event on Virtual Reality next month at 5:00

In [80]:
import csv
import pandas as pd  
import datefinder
import datetime
from dateutil.parser import parse
import json

In [81]:
event_types = ["Birthday", "Meeting", "Seminar", "Party", "Anniversary","Marriage", "Appointment", "Meet", "Workshop", "career fair"]
event_dates = ["Tomorrow", "Today", "Day After Tomorrow", "Next Month", "Next Week"]

In [84]:
def getDate(str):
    now = datetime.datetime.now()
    datetime.datetime.now() + datetime.timedelta(days=1)
    switcher = { 
        "today" : now , 
        "tomorrow": now + datetime.timedelta(days=1), 
        "day after tommorow": now + datetime.timedelta(days=2), 
        "next week": now + datetime.timedelta(days=7),
        "next month": now + datetime.timedelta(days=30)
    }
    return switcher.get(str, "Not a valid date")

csvFile = pd.read_csv('CSV_NAME.csv') 
messages = csvFile['Message_body']
senders = csvFile['Sender']
subjects = csvFile['Subject']
empevents = []
empdates = []
event_list = []
msg = []
msg_labels = []

msg_counter = 0
for index,row in messages.iteritems():
    flag_ml = False
    i = 0
    event_dict = {}
    str1 = ''.join(row)
    msg.append(str1)
    for event_type in event_types:
        if event_type.lower() in str1.lower(): 
            label = 'yes'
            msg_labels.append('yes')
            flag = False
            for event_date in event_dates:
                if event_date.lower() in str1.lower():
                    convertedDate = getDate(event_date.lower())
                    json_event_date = convertedDate.date()
                    matches = datefinder.find_dates(str1)
                    for match in matches:
                        if match.time():
                            if i == 0:
                                json_event_startTime = match.time()
                            elif i == 1:
                                json_event_endTime = match.time()
                        i+=1
                    flag = True
                    break
            if not flag :
                matches = datefinder.find_dates(str1)

                # a = list(matches)[:]
                # print(len(a))
                for match in matches:
                    if i == 0:
                        json_event_date = match.date()
                        json_event_startTime = match.time()
                    elif i == 1:
                        json_event_endTime = match.time()
                    i+=1

            json_event_type = event_type
            json_event_sender = senders.iloc[msg_counter]
            json_event_subject = subjects.iloc[msg_counter]
            if i == 0:
                json_event_startTime = "00:00:00"
                json_event_endTime = "00:00:00"
            elif i == 1:
                json_event_endTime = "00:00:00"
            event_dict["event_type"] = json_event_type
            event_dict["event_date"] = str(json_event_date)
            event_dict["event_startTime"] = str(json_event_startTime)
            event_dict["event_endTime"] = str(json_event_endTime)
            event_dict["event_sender"] = json_event_sender
            event_dict["event_subject"] = json_event_subject
            event_list.append(event_dict)
            flag_ml = True
            break
    if not flag_ml:
        label = 'no'
        msg_labels.append('no')
    with open('labeledEmails.csv', 'a', newline='') as csvfile1:
        fieldnames = ['Email','Label']
        writer = csv.DictWriter(csvfile1, fieldnames=fieldnames)
        writer.writerow({'Email':str1, 'Label':label})

        
with open('test.json', 'w') as fout:
    json.dump(event_list , fout)

print(len(msg))
print(len(msg_labels))
for empdate in empdates:
    print(empdate.date())
for event in event_list:
    print(event)

13
13
{'event_type': 'Meet', 'event_date': '2019-06-20', 'event_startTime': '11:00:00', 'event_endTime': '00:00:00', 'event_sender': 'wamique Ansari <wamique.ansari@gmail.com>', 'event_subject': 'CMPE272 Class Lecture today at 5:00 PM'}
{'event_type': 'Anniversary', 'event_date': '2019-04-11', 'event_startTime': '13:03:00', 'event_endTime': '00:00:00', 'event_sender': 'wamique Ansari <wamique.ansari@gmail.com>', 'event_subject': 'CMPE272 Class Lecture today at 5:00 PM'}
{'event_type': 'career fair', 'event_date': '2019-05-30', 'event_startTime': '13:00:00', 'event_endTime': '00:00:00', 'event_sender': 'wamique Ansari <wamique.ansari@gmail.com>', 'event_subject': 'CMPE272 Class Lecture today at 5:00 PM'}
{'event_type': 'Birthday', 'event_date': '2019-05-10', 'event_startTime': '16:00:00', 'event_endTime': '00:00:00', 'event_sender': 'wamique Ansari <wamique.ansari@gmail.com>', 'event_subject': 'CMPE272 Class Lecture today at 5:00 PM'}
{'event_type': 'Meeting', 'event_date': '2019-05-09'

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

In [73]:
#converting the string into vector to pass into our classifier
def transform_to_features(data):
    from sklearn.feature_extraction.text import CountVectorizer
    vectorizer = CountVectorizer(
        analyzer='word',
        lowercase=False,
    )
    features = vectorizer.fit_transform(
        data
    )
    features_nd = features.toarray()
    return features_nd

In [74]:
#Splitting the data into Train,Test with Train as 80%
def train_then_build_model(data_labels, features_nd, data):
    from sklearn.model_selection import train_test_split
    # TODO : set training % to 80%.
    X_train, X_test, y_train, y_test = train_test_split(
        features_nd,
        data_labels,
        train_size=0.80,
        random_state=1234)
    X_traindata, X_testdata, y_traindata, y_testdata = train_test_split(
        data,
        data_labels,
        train_size=0.80,
        random_state=1234)
    
#importing the MultinomialNB from sklearn and fitting the train and test data sets
    from sklearn.naive_bayes import MultinomialNB
    mnb = MultinomialNB()
    mnb.fit(X_train, y_train)
    predmnb = mnb.predict(X_test)
    for i in range(len(X_testdata)):
        print("Msg: ", X_testdata[i],"Predicted Class: ", predmnb[i])
        
    print("Confusion Matrix for Multinomial Naive Bayes:")
    print(confusion_matrix(y_test, predmnb))
    print("Score:", round(accuracy_score(y_test, predmnb) * 100, 2))
    print("Classification Report:")
    print(classification_report(y_test, predmnb))

In [75]:
def process():
    features_nd = transform_to_features(msg)
    train_then_build_model(msg_labels, features_nd, msg)

In [76]:
process()

Msg:  [<p>You are invited to attend my marriage on Thursday April 11, 2019 from 1:03
PM to 2:03 PM
</p>] Predicted Class:  yes
Msg:  [<p>Dear Spartans,

Mingle with U.S. and international students while enjoying snacks and
refreshments with Coffee Night. Fun new themes every week! Open to all SJSU
students.

See you all at International House on Tue, May 21, 2019 at 8:30 PM to 10:30
PM.

Regards
Coffee Team
</p>] Predicted Class:  no
Confusion Matrix for Multinomial Naive Bayes:
[[1 0]
 [0 1]]
Score: 100.0
Classification Report:
              precision    recall  f1-score   support

          no       1.00      1.00      1.00         1
         yes       1.00      1.00      1.00         1

   micro avg       1.00      1.00      1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)
